In [1]:
import os
import pathlib
import random

import csv
import wget
import cv2
import math
import numpy as np
import pandas as pd
import scipy as scp 
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, widgets

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from skimage.feature import hog
from skimage import data, exposure

In [ ]:
DATA_PATH = f"{os.environ['HOME']}/Documents/TU Berlin/10 SS22/BGA II/UBs/data"
cl = [random.randint(0, 42) for _ in range(2)]
CLASSES = [str('%05d' % i) for i in cl]
images_folders = ['GTSRB_Final_Training_Images/GTSRB/Final_Training/Images',
    'GTSRB_Final_Test_Images/GTSRB/Final_Test/Images', # + Classes Folder
    ]
hog_paths = ['GTSRB_Final_Test_HOG/GTSRB/Final_Test/HOG/HOG_01',
    'GTSRB_Final_Test_HOG/GTSRB/Final_Test/HOG/HOG_02',
    'GTSRB_Final_Test_HOG/GTSRB/Final_Test/HOG/HOG_03',
    'GTSRB_Final_Training_HOG/GTSRB/Final_Training/HOG/HOG_01', # + Classes Folder
    'GTSRB_Final_Training_HOG/GTSRB/Final_Training/HOG/HOG_02', # + Classes Folder
    'GTSRB_Final_Training_HOG/GTSRB/Final_Training/HOG/HOG_03', # + Classes Folder
    ]
TRAIN_PATH = os.path.join(DATA_PATH, "GTSRB_Final_Training_Images/GTSRB/Final_Training/Images")
TEST_PATH = os.path.join(DATA_PATH, "GTSRB_Final_Test_Images/GTSRB/Final_Test/Images")
print(CLASSES)

In [4]:
TRAIN_DATA = []
LABEL_DATA = []
dict_mapping = dict().fromkeys([int(i) for i in CLASSES], [])
for cl in CLASSES:
    tmp = os.path.join(DATA_PATH, 'GTSRB_Final_Training_Images/GTSRB/Final_Training/Images', cl)
    if os.path.exists(tmp):
        for ele in os.listdir(tmp):
            if ".csv" not in ele:
                TRAIN_DATA.append([os.path.join(tmp, ele), int(cl)])
                LABEL_DATA.append(int(cl))
                dict_mapping[int(cl)].append(os.path.join(tmp, ele))
    else:
        print(cl, tmp)

TRAIN_DATA = np.array(TRAIN_DATA)

In [5]:
X_train, X_test = train_test_split(TRAIN_DATA, test_size=0.25)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(X_train[:,1])
X_train = np.array(X_train[:,0])
y_test = np.array(X_test[:,1])
X_test = np.array(X_test[:,0])

In [9]:
def hoging(file_path, orient: int = 8, px_cell = (2,2), cell_block = (1,1), rescale = (0,10)):
    color_img = cv2.imread(file_path, cv2.IMREAD_COLOR)
    
    image = cv2.cvtColor(color_img,cv2.COLOR_BGR2GRAY)
    
    # image = cv2.medianBlur(img_gray,5)

    _, hog_image = hog(image, orientations=orient, pixels_per_cell=px_cell,
                        cells_per_block=cell_block, visualize=True)
    # , channel_axis=None

    # Rescale histogram for better display
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=rescale)

    return hog_image, hog_image_rescaled

In [10]:
def show_img_form(idx):

    h_img, h_rsc_img = hoging(X_train[idx], orient=8, px_cell=(2,2), cell_block=(1,1), rescale=(0,10))
    # print(h_img.shape)
    # print(h_rsc_img.shape)
    image = plt.imread(X_train[idx])

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(8, 4), sharex=True, sharey=True)

    ax1.axis('off')
    ax1.imshow(image, cmap=plt.cm.gray)
    ax1.set_title('Input image')

    ax2.axis('off')
    ax2.imshow(h_img, cmap=plt.cm.gray)
    ax2.set_title('HOG')

    ax3.axis('off')
    ax3.imshow(h_rsc_img, cmap=plt.cm.gray)
    ax3.set_title('HOG - Rescaled')

    plt.show()

In [11]:
interact(show_img_form, idx=widgets.IntSlider(min=2, max=len(X_train)-1, step=1, value=0));

interactive(children=(IntSlider(value=2, description='idx', max=1709, min=2), Output()), _dom_classes=('widget…

In [16]:
def eigen_verkehrsschild(data_, n_comp=0.5):
    eigverkehrsschild = PCA(n_components=n_comp)
    fit_data = []
    shapes = []
    for item in data_:
        h_img, _ = hoging(item, orient=1, px_cell=(3,2), cell_block=(1,1), rescale=(0,10))
        # shapes.append(h_img.shape)
        if len(h_img.shape) > 2:
            print(item)
            continue
        else:
            fit_data.append(h_img[:15, :15])

    eigverkehrsschild.fit(fit_data)
    return eigverkehrsschild

In [17]:
EIGVERKEHRSSCHILDEN = eigen_verkehrsschild(X_train, n_comp=0.5)

ValueError: Found array with dim 3. Estimator expected <= 2.

In [ ]:
def show_eigen_verkehrsschild(idx):
    image = plt.imread(X_train[idx])

    fig, axes = plt.subplots(1, 6, figsize=(8, 4), sharex=True, sharey=True)

    for i, ax in enumerate(axes.flat):
        if i == 0:
            ax.axis('off')
            ax.imshow(image, cmap=plt.cm.gray)
            ax.set_title('Input image')
        else:
            ax.imshow(EIGVERKEHRSSCHILDEN.components_[i].reshape(112,92),cmap="gray")
    plt.show()

In [18]:
# np.unique(EIGVERKEHRSSCHILDEN)
EIGVERKEHRSSCHILDEN = set(EIGVERKEHRSSCHILDEN)

In [20]:
EIGVERKEHRSSCHILDEN

{(25, 26),
 (25, 27),
 (26, 25),
 (26, 26),
 (26, 27),
 (26, 28),
 (26, 29),
 (26, 30),
 (27, 25),
 (27, 26),
 (27, 27),
 (27, 28),
 (27, 29),
 (27, 30),
 (27, 31),
 (28, 26),
 (28, 27),
 (28, 28),
 (28, 29),
 (28, 30),
 (28, 31),
 (28, 32),
 (29, 26),
 (29, 27),
 (29, 28),
 (29, 29),
 (29, 30),
 (29, 31),
 (29, 32),
 (29, 33),
 (30, 28),
 (30, 29),
 (30, 30),
 (30, 31),
 (30, 32),
 (30, 33),
 (30, 34),
 (31, 27),
 (31, 28),
 (31, 29),
 (31, 30),
 (31, 31),
 (31, 32),
 (31, 33),
 (31, 34),
 (31, 35),
 (32, 27),
 (32, 29),
 (32, 30),
 (32, 31),
 (32, 32),
 (32, 33),
 (32, 34),
 (32, 35),
 (32, 36),
 (33, 30),
 (33, 31),
 (33, 32),
 (33, 33),
 (33, 34),
 (33, 35),
 (33, 36),
 (33, 37),
 (34, 30),
 (34, 31),
 (34, 32),
 (34, 33),
 (34, 34),
 (34, 35),
 (34, 36),
 (34, 37),
 (34, 38),
 (34, 39),
 (35, 29),
 (35, 30),
 (35, 32),
 (35, 33),
 (35, 34),
 (35, 35),
 (35, 36),
 (35, 37),
 (35, 38),
 (35, 39),
 (35, 40),
 (36, 31),
 (36, 32),
 (36, 33),
 (36, 34),
 (36, 35),
 (36, 36),
 (36, 37),